In [1]:
import argparse

import torch
import timm
import torchvision
from torch.utils.data import DataLoader

from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint

from misc import collate_fn, Augment_v2
from model import MoCo


In [2]:
model = timm.create_model('vit_base_patch16_224', pretrained=False, img_size=224)

In [35]:

import torch.nn.functional as F

q1 = torch.tensor([(1,2,3),(1,2,3),(1,2,3)], dtype=torch.float32)
z1 = torch.tensor([(-4,1,2),(0,5,3),(1,2,3)], dtype=torch.float32)
q1.shape
# z1 = torch.tensor((2,2,3))

# q1 = torch.randn(10,10)
# z1 = torch.randn(10,10)
p = F.normalize(q1, dim=1)
z = F.normalize(z1, dim=1)

2 - 2 * (p * z).sum(dim=1).mean()

tensor(0.5972)

In [40]:
(p * z).sum(dim=1)

tensor([0.2333, 0.8709, 1.0000])

In [ ]:
torch.matmul(p, z.T)

tensor([[0.2333, 0.8709, 1.0000],
        [0.2333, 0.8709, 1.0000],
        [0.2333, 0.8709, 1.0000]])

In [37]:

p = F.normalize(p, dim=1)
z = F.normalize(z, dim=1)
2 - 2 * (p * z).sum(dim=1).mean()

tensor(0.5972)